In [8]:
import atomium
import math
import mdtraj as md
import nglview
import pybel as bel
import os
from tqdm import tqdm
from rmsd import *
import numpy as np
from pmapper.pharmacophore import Pharmacophore as Pharma
from rdkit import Chem
from rdkit.Chem import AllChem

In [9]:
input_folder = "SI/DockingInput/Ligands_pdb/"
output_folder = "SI/DockingResults/Interface/LigandsOut_0/Ligands_pdb/"

In [10]:
def trans_rot(P,Q):
    
    p = P.mean(axis = 0)
    q = Q.mean(axis = 0)
    T = q-p   #Translation vector
    P += T
    
    R = kabsch(P,Q)   # Rotation matrix
    return np.concatenate([T, np.ndarray.flatten(R)])
    

In [11]:
def get_XY():
    
    X = []
    Y = []
    
    for file in tqdm(os.listdir(input_folder)):
        try:
            Initial_molecule = Chem.MolFromPDBFile(input_folder+file)
            Final_molecule = Chem.MolFromPDBFile(output_folder+file)
            
        except:
            continue
        
        Chem.rdMolTransforms.CanonicalizeMol(Initial_molecule)   # Aligns principal axes with x, y, z
        Initial_coords = Initial_molecule.GetConformers()[0].GetPositions()
        
        Final_coords = Final_molecule.GetConformers()[0].GetPositions()
        
        p = Pharma()
        p.load_from_mol(Initial_molecule)
        bitset = p.get_fp(min_features=4, max_features=4, nbits=2048, activate_bits=2) # Can change these parameters to possibly affect accuracy
        
        x = [False]*2049
        for i in bitset: x[i] = True
            
        X.append(x)  
        Y.append(trans_rot(Initial_coords, Final_coords))
        
    return (X,Y)
    

In [12]:
def main():
    X,Y = get_XY()
    print(np.array(X))
    print(np.array(Y))

In [13]:
main()

100%|██████████| 122/122 [00:05<00:00, 21.87it/s]

[[False False False ... False False False]
 [False False False ... False False False]
 [ True False False ... False  True False]
 ...
 [False False False ... False False False]
 [False False False ... False  True False]
 [False False False ... False False False]]
[[-4.5109     -2.5648      3.1852     ...  0.05018715 -0.36095939
   0.93123014]
 [-4.98614286 -3.57992857  2.93885714 ... -0.77497071 -0.21146856
  -0.59556817]
 [-4.45963158 -1.40357895  2.27921053 ... -0.24386837  0.38187985
   0.89145723]
 ...
 [-4.66710526 -3.06563158  2.57263158 ... -0.90571724  0.31700376
  -0.28139811]
 [-4.70189474 -3.31536842  3.19057895 ... -0.59493707  0.41177816
   0.69028156]
 [-4.5474     -3.0524      2.4022     ...  0.01610437 -0.13075713
   0.99128362]]
